In [55]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [56]:
product_type = "frozen"
num_time_periods = 7

In [57]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [58]:
# constants definitions
if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000

In [59]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [60]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [61]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [62]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [63]:
idx = pd.MultiIndex.from_product([forecast_data['ARTICLE_ID'].unique(), pd.date_range('2022-01-01', '2022-12-31', freq='D')], names=['ARTICLE_ID', 'DATE'])


In [71]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start='2022-06-13', end='2022-06-18', freq='D')}).astype(str)
# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()
demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()

In [74]:
# model object
m = gp.Model()

# decision variables
# Xit
orders = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Yit
ordered_boolean = m.addVars(items, time_periods, vtype=gp.GRB.BINARY, lb=0)
# Sit
storage_used = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Zt
buffer_storage_used = m.addVars(time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Dit
clearances = m.addVars(items, time_periods, vtype=gp.GRB.INTEGER, lb=0)
# Xit1t2
orders_disaggregated = m.addVars(items, time_periods, time_periods, vtype=gp.GRB.INTEGER, lb=0)

# objective function
ordering_cost_per_tu_objective = gp.quicksum(ordering_cost_per_tu[item] * orders[item, time_period] for item in items for time_period in time_periods)
ordering_cost_fixed_objective = gp.quicksum(ordering_cost_fixed[item] * ordered_boolean[item, time_period] for item in items for time_period in time_periods)
buffer_storage_objective = gp.quicksum(buffer_cost * buffer_storage_used[time_period] for time_period in time_periods)
clearance_objective = gp.quicksum(clearing_cost_per_cu[item] * clearances[item, time_period] for item in items for time_period in time_periods)

m.setObjective(ordering_cost_per_tu_objective + ordering_cost_fixed_objective + buffer_storage_objective, sense=gp.GRB.MINIMIZE)

# constraints
# demand satisfaction
for item in demand.keys():
    for time_period_iterator, time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr(orders[item, time_period] * cu_per_tu[item] - clearances[item, time_period]>= demand[item][time_period],name="demand constraint_" + str(time_period))
        else:
            m.addConstr((orders[item, time_period] * cu_per_tu[item]) + storage_used[item, time_periods[time_period_iterator-1]] - clearances[item, time_period]>= demand[item][time_period],name="demand constraint_" + str(time_period))

# inventory volume constraint
for time_period_iterator, time_period in enumerate(time_periods):
    if(time_period_iterator==0):
        m.addConstr(gp.quicksum(volume_per_cu[item] * ((cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])
    else:
        m.addConstr(gp.quicksum(volume_per_cu[item] * (storage_used[item, time_periods[time_period_iterator-1]] + (cu_per_tu[item] * (orders[item, time_period]))) for item in items) <= warehouse_volume + buffer_storage_used[time_period])

# # min/max constraints (linking too)
for item in demand.keys():
    for time_period in time_periods:
        m.addConstr(orders[item, time_period] >= minimum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # if((not math.isnan(maximum_order_quantity_tu[item]))):
        #     m.addConstr(orders[item, time_period] <= maximum_order_quantity_tu[item] * ordered_boolean[item, time_period])
        # else:
        #     m.addConstr(orders[item, time_period] <= default_max_order * ordered_boolean[item, time_period])

# current inventory constraint
for item in demand.keys():
    for time_period_iterator,time_period in enumerate(time_periods):
        if(time_period_iterator==0):
            m.addConstr(storage_used[item, time_periods[time_period_iterator]] == (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])
        else:
            m.addConstr(storage_used[item, time_periods[time_period_iterator]] == storage_used[item, time_periods[time_period_iterator-1]] + (orders[item, time_period] * cu_per_tu[item]) - demand[item][time_period])

# clearance constraint
for item in demand.keys():
    life = shelf_life[item]
    for time_period in time_periods[:-life]:
        m.addConstr(clearances[item, time_period] <= (orders[item, time_period] * cu_per_tu[item]) - (gp.quicksum(orders_disaggregated[item, time_period, t2] for t2 in time_periods[time_period_iterator:time_period_iterator+life])))

m.setParam(gp.GRB.Param.Heuristics, 0) 
m.optimize()

Set parameter Heuristics to value 0
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6288 rows, 25146 columns and 18827 nonzeros
Model fingerprint: 0x6a0e798f
Variable types: 0 continuous, 25146 integer (2514 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+01]
  Objective range  [1e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 6288 rows and 25146 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 4966.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.966200000000e+03, best bound 4.966200000000e+03, gap 0.0000%
